In [7]:
import string 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import json
try: import spacy   
except: import spacy
from spacy import displacy
name = 'finance_course1'
JSON_path = '/home/ubuntu/Questions_generation/Financial Markets Course 2.json'


with open(JSON_path) as f:
  result = json.load(f)

sections = [s['summary']['text'] if 'summary' in s.keys() and 'text' in s['summary'].keys() else None for k,s in result['sections'].items() ]
org_text = [s['original']['text'] if 'original' in s.keys() and 'text' in s['original'].keys() else None  for k,s in result['sections'].items() ]


with open('/home/ubuntu/Questions_generation/outputs/03_01_23_finance_course2/questions.txt') as f:
    text = f.read()
text = text.replace('\nQ:','.\n ').replace('\nA:',' ').replace ('--------------------------------------------------','').split('.')

text = org_text
#text = "what year Israel was established? 1948"
nlp = spacy.load("en_core_web_sm")
if isinstance(text,list):
    doc= nlp("\n".join(text))
else:
    doc =nlp(text)

#displacy.render(doc, style="ent")
#displacy.render(doc, style='dep')

def remove_stop_words(text):
    tokens = word_tokenize(text)
    punct_or_stopwords = set([*stopwords.words('english'), *string.punctuation])
    return " ".join([token for token in tokens if token.lower() not in punct_or_stopwords])
def only_ents(text,doc):
    ans_idx = text.find('?')+2
    filtered_ans = remove_stop_words(text[ans_idx:])
    print(filtered_ans)
    return any([filtered_ans.strip() == cur_ent.text for cur_ent in doc.ents])
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)
'''
text = "what year Israel was established? few years after 1948."
doc =nlp(text)

print(only_ents(text,doc))
displacy.render(doc, style='dep')
displacy.render(nlp('few years after 1948'), style='dep',jupyter=True)'''

today 317 322 DATE
2007 430 434 DATE
the Great Depression of the 1930's 529 563 EVENT
one 656 659 CARDINAL
around 2000 1596 1607 DATE
2000 1709 1713 DATE
2003 1751 1755 DATE
2007 2026 2030 DATE
the United Kingdom 2121 2139 GPE
Northern Rock 2141 2154 GPE
1930 2192 2196 DATE
the United States 2261 2278 GPE
today 2680 2685 DATE
Tomorrow 3386 3394 DATE
today 3459 3464 DATE
two 3824 3827 CARDINAL
One 3906 3909 CARDINAL
1600 4384 4388 DATE
the year 1600 4430 4443 DATE
0.5 4480 4483 CARDINAL
millions 4969 4977 CARDINAL
millions 5230 5238 CARDINAL
two weeks 6393 6402 DATE
Elan Fuld 6891 6900 ORG
one 6902 6905 CARDINAL
ROBERT SHILLER 7143 7157 PERSON
a year 7306 7312 DATE
months 7329 7335 DATE
a day 7352 7357 DATE
these months 7381 7393 DATE
monthly 7410 7417 DATE
these months 7448 7460 DATE
the first month 7470 7485 DATE
second month 7501 7513 DATE
two 7524 7527 CARDINAL
3 7572 7573 CARDINAL
month three 7603 7614 DATE
the beginning of the month 7638 7664 DATE
1 7769 7770 CARDINAL
less than mi

'\ntext = "what year Israel was established? few years after 1948."\ndoc =nlp(text)\n\nprint(only_ents(text,doc))\ndisplacy.render(doc, style=\'dep\')\ndisplacy.render(nlp(\'few years after 1948\'), style=\'dep\',jupyter=True)'

In [4]:
class StringDistance:
    def distance(self, s0, s1):
        raise NotImplementedError()

class StringSimilarity:

    def similarity(self, s0, s1):
        raise NotImplementedError()

class NormalizedStringDistance(StringDistance):

    def distance(self, s0, s1):
        raise NotImplementedError()

class NormalizedStringSimilarity(StringSimilarity):

    def similarity(self, s0, s1):
        raise NotImplementedError()

class MetricStringDistance(StringDistance):

    def distance(self, s0, s1):
        raise NotImplementedError()
class Levenshtein(MetricStringDistance):

    def distance(self, s0, s1):
        if s0 is None:
            raise TypeError("Argument s0 is NoneType.")
        if s1 is None:
            raise TypeError("Argument s1 is NoneType.")
        if s0 == s1:
            return 0.0
        if len(s0) == 0:
            return len(s1)
        if len(s1) == 0:
            return len(s1)

        v0 = [0] * (len(s1) + 1)
        v1 = [0] * (len(s1) + 1)

        for i in range(len(v0)):
            v0[i] = i

        for i in range(len(s0)):
            v1[0] = i + 1
            for j in range(len(s1)):
                cost = 1
                if s0[i] == s1[j]:
                    cost = 0
                v1[j + 1] = min(v1[j] + 1, v0[j + 1] + 1, v0[j] + cost)
            v0, v1 = v1, v0

        return v0[len(s1)]
  
class NormalizedLevenshtein(NormalizedStringDistance, NormalizedStringSimilarity):

    def __init__(self):
        self.levenshtein = Levenshtein()

    def distance(self, s0, s1):
        if s0 is None:
            raise TypeError("Argument s0 is NoneType.")
        if s1 is None:
            raise TypeError("Argument s1 is NoneType.")
        if s0 == s1:
            return 0.0
        m_len = max(len(s0), len(s1))
        if m_len == 0:
            return 0.0
        return self.levenshtein.distance(s0, s1) / m_len

    def similarity(self, s0, s1):
        return 1.0 - self.distance(s0, s1)

normalized_levenshtein = NormalizedLevenshtein()


def have_common_words(str1, str2):
    list1 = str1.split()
    list2 = str2.split()
    list1_as_set = set(list1)
    intersection = list1_as_set.intersection(list2)
    if intersection:
        return True
    return False

def is_abbreviation(phrase1,phrase2):
    phrase1 = phrase1.strip().lower()
    phrase2 = phrase2.strip().lower()
    if len(phrase1) < len(phrase2):
        initials, full_phrase = phrase1, phrase2
    else:
        initials, full_phrase = phrase2, phrase1
    initials = initials.replace(".","")
    full_phrase_list = [word.strip() for word in full_phrase.split() if word.strip() not in stopwords.words('english')]
    if len(initials) < 2 or len(full_phrase_list) < 2:
        return False

    # option #1: 
    # if len(initials) != len(full_phrase_list):
    #     return False
    # for idx, char in enumerate(initials):
    #     if char != full_phrase_list[idx][0]:
    #         return False
    # return True
    # option #2:
    if initials[0] == full_phrase_list[0][0] and initials[1] == full_phrase_list[1][0]:
        return True
    return False

def filter_distractors(word, candidates, n=5, threshold = 0.7):
    distractors = [word]
    for candidate in candidates:
        if len(distractors) >= n+1:
            return distractors[1:]
        x = []
        for distractor in distractors:
            if not have_common_words(candidate, distractor) \
            and candidate.lower() not in edits(distractor.lower()) \
            and distractor.lower() not in edits(candidate.lower()) \
            and not is_abbreviation(distractor,candidate) \
            and normalized_levenshtein.distance(distractor.lower(),candidate.lower()) > threshold: 
                x.append(True)
            else:
                x.append(False)
        if all(x):
            distractors.append(candidate) 
    return distractors[1:]

def edits(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz '+string.punctuation
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)


In [1]:
import string 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import re
def model_loader(kind, name, load):
  name = kind + ":" + name
  if name not in pre_loaded:
    pre_loaded[name] = load()
  return pre_loaded[name]

from sense2vec import Sense2Vec
pre_loaded = {}
s2v = model_loader(
    'sense2vec',
    "s2v",
    lambda : Sense2Vec().from_disk("/home/ubuntu/Questions_generation/s2v_reddit_2019_lg") # large 
)

def filter_distractors(word, candidates, n=5, threshold = 0.7):
    distractors = [word]
    for candidate in candidates:
        if len(distractors) >= n+1:
            return distractors[1:]
        x = []
        for distractor in distractors:
            if not have_common_words(candidate, distractor) \
            and candidate.lower() not in edits(distractor.lower()) \
            and distractor.lower() not in edits(candidate.lower()) \
            and not is_abbreviation(distractor,candidate) \
            and normalized_levenshtein.distance(distractor.lower(),candidate.lower()) > threshold: 
                x.append(True)
            else:
                x.append(False)
        if all(x):
            distractors.append(candidate) 
    return distractors[1:]

def find_distractors_s2v(term, n=5):
    try:
        candidates = [
                    result[0].split("|")[0].replace("_"," ")
                    for result in s2v.most_similar(term, n=30)
                    if result[0].split("|")[1] == term.split("|")[1]
                    and result[0] == s2v.get_best_sense(result[0].split("|")[0].replace("_"," "))
        ]
        # print(candidates)
        
        distractors = filter_distractors(term.split("|")[0].replace("_"," "), candidates, n=n ,threshold = 0.7)

        if distractors:
            return distractors
    except ValueError as e:
        return
find_distractors_s2v('2002|DATE',10)

[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/home/ubuntu/miniforge3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'have_common_words' is not defined

In [13]:
import re
abrv = 'RNN'
text = 'what are RNN?. RNN!'
long_form_abbr ='recurrent neural network'

print(text)
pattern = re.compile(r''+long_form_abbr+r'\s*\(\s*'+abrv+r'\s*\)', re.IGNORECASE)
text2 = re.sub(pattern,abrv,text)
print('compress:',text2)
target =lambda abrv,long_form_abrv: f'{long_form_abrv} ({abrv})'
pattern = re.compile(r'(^|\s|\.)('+abrv+')(\W)', re.MULTILINE | re.IGNORECASE)
text = re.sub(pattern,r'\1'+target(abrv,long_form_abbr)+ r'\3' ,  text2, count=7) 

pattern = re.compile(r'(^|\s|\.)('+abrv+')(\W)', re.MULTILINE | re.IGNORECASE)

print(text)
pattern = re.compile(r''+long_form_abbr+r'\s*\(\s*'+abrv+r'\s*\)'+r'\s*\(\s*'+abrv+r'\s*\)',re.MULTILINE | re.IGNORECASE)
text = re.sub(pattern,r'\1'+target(abrv,long_form_abbr)+ r'\3' ,  text2, count=7) 

what are RNN?. (RNN!
compress: what are RNN?. (RNN!
what are recurrent neural network (RNN)?. (RNN!


In [30]:
text2 = " Stog (STD) (std)"
abrv ='STD'
pattern = re.compile(r'(^|\s|\.|\,)(\(\s*'+abrv+r'\s*\))\s*\(\s*'+abrv+r'\s*\)',re.MULTILINE | re.IGNORECASE)
text = re.sub(pattern,r'\1\2' ,  text2, count=7) 
text

' Stog (STD)'

In [21]:
r''+long_form_abbr+r'\s* \( \s*'+abrv+r' \s* \)',r"recurrent neural network\s* \( \s*RNN\s*\)"

('recurrent neural network\\s* \\( \\s*RNN \\s* \\)',
 'recurrent neural network\\s* \\( \\s*RNN\\s*\\)')

In [1]:

from sense2vec import Sense2Vec
s2v = Sense2Vec().from_disk("/home/ubuntu/Questions_generation/s2v_reddit_2019_lg")
import spacy
nlp = spacy.load("en_core_web_sm")
#s2v = nlp.add_pipe("sense2vec")
#s2v.from_disk("/path/to/s2v_reddit_2015_md")
print(s2v.most_similar('Apple|ORG', n=30))
doc = nlp("A sentence about Joe.")
#assert doc[3].text == "Apple"
freq = doc[3]._.s2v_freq
vector = doc[3]._.s2v_vec
most_similar = doc[3]._.s2v_most_similar(3)

/home/ubuntu/miniforge3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ValueError: Can't find key Aplle|ORG in table

In [2]:
print(s2v.most_similar('Apple|ORG', n=30))

[('apple|ORG', 0.8275), ('apple|PROPN', 0.7979), ('Qualcomm|ORG', 0.7365), ('Samsung|ORG', 0.7224), ('Apples|ORG', 0.7143), ('A1181|PROPN', 0.7068), ('iPhones|ORG', 0.7052), ('BlackBerry|ORG', 0.7037), ('apple|NOUN', 0.6978), ('Blackberry|ORG', 0.6959), ('Tim_Cook|PERSON', 0.6955), ('iPhone|ORG', 0.6881), ('Apple|PROPN', 0.6867), ('MacBooks|ORG', 0.6662), ('apple_products|NOUN', 0.6661), ('non-Apple|ORG', 0.6635), ('Microsoft|ORG', 0.6623), ('other_phone_manufacturers|NOUN', 0.6615), ('iOS|CARDINAL', 0.6597), ('apple_product|NOUN', 0.6589), ('Apple|PRODUCT', 0.6562), ('OnePlus|PERSON', 0.6558), ('APPLE|ORG', 0.6552), ('only_apple|NOUN', 0.6544), ('Macs|PRODUCT', 0.6536), ('A1343|PROPN', 0.6531), ('phone_manufacturer|NOUN', 0.6522), ('app_developers|NOUN', 0.6522), ('Android|ORG', 0.6481), ('iOS|ORG', 0.6463)]


In [1]:
doc = nlp("A sentence about Joe.")
#assert doc[3].text == "Apple"
freq = doc[3]._.s2v_freq
vector = doc[3]._.s2v_vec
most_similar = doc[3]._.s2v_most_similar(3)

NameError: name 'nlp' is not defined

In [13]:
from transformers import GPTNeoXForCausalLM, GPTNeoXTokenizerFast

model = GPTNeoXForCausalLM.from_pretrained("EleutherAI/gpt-neox-20b")
tokenizer = GPTNeoXTokenizerFast.from_pretrained("EleutherAI/gpt-neox-20b")

prompt = "GPTNeoX20B is a 20B-parameter autoregressive Transformer model developed by EleutherAI."

input_ids = tokenizer(prompt, return_tensors="pt").input_ids

gen_tokens = model.generate(
    input_ids,
    do_sample=True,
    temperature=0.9,
    max_length=100,
)
gen_text = tokenizer.batch_decode(gen_tokens)[0]

Downloading: 100%|██████████| 617/617 [00:00<00:00, 548kB/s]
Downloading: 100%|██████████| 57.7k/57.7k [00:00<00:00, 34.4MB/s]
Downloading: 100%|██████████| 926M/926M [00:10<00:00, 87.5MB/s] 
Downloading: 100%|██████████| 910M/910M [00:10<00:00, 84.4MB/s] 
Downloading: 100%|██████████| 910M/910M [00:10<00:00, 83.6MB/s] 
Downloading: 100%|██████████| 910M/910M [00:12<00:00, 74.2MB/s] 
Downloading: 100%|██████████| 910M/910M [00:12<00:00, 70.8MB/s] 
Downloading: 100%|██████████| 910M/910M [00:16<00:00, 53.6MB/s] 
Downloading: 100%|██████████| 910M/910M [00:12<00:00, 72.4MB/s] 
Downloading: 100%|██████████| 910M/910M [00:12<00:00, 72.9MB/s] 
Downloading: 100%|██████████| 910M/910M [00:10<00:00, 84.8MB/s] 
Downloading: 100%|██████████| 910M/910M [00:13<00:00, 66.8MB/s] 
Downloading: 100%|██████████| 910M/910M [00:10<00:00, 85.5MB/s] 
Downloading: 100%|██████████| 910M/910M [00:11<00:00, 81.5MB/s] 
Downloading: 100%|██████████| 910M/910M [00:10<00:00, 84.3MB/s] 
Downloading: 100%|██████████

In [6]:
nlp = spacy.load("en_core_web_sm")
s2v = Sense2Vec().from_disk("/home/ubuntu/Questions_generation/s2v_old")

#s2v = nlp.add_pipe("sense2vec")
doc = nlp("A sentence about Facebook and Google.")
for ent in doc.ents:
    assert ent._.in_s2v
    most_similar = ent._.s2v_most_similar(3)

AttributeError: 'NoneType' object has no attribute 'cfg'

In [ ]:
dir = '/home/ubuntu/Questions_generation/outputs/03_01_23_terror_cyber/'
name = 'terror'
JSON_path = '/home/ubuntu/Questions_generation/Terrorism Enters Cyberspace.json'
full_q_file = 'questions_full.txt'
q_file_path = 'questions.txt'
import json
with open(JSON_path) as f:
  result = json.load(f)
import spacy
sections = [s['summary']['text'] if 'summary' in s.keys() and 'text' in s['summary'].keys() else None for k,s in result['sections'].items() ]
org_text = [s['original']['text'] if 'original' in s.keys() and 'text' in s['original'].keys() else None  for k,s in result['sections'].items() ]

output = []
import pandas as pd
import numpy as np
questions_df = pd.read_pickle(dir+'GQ+QA+GQ.pickle')
for i in range(max(questions_df.section_n)):
    section_output = {'question':[],'answer':[],'score':[]}
    for chunk in np.unique(questions_df.loc[questions_df.section_n==i,'section_n_chunk']):
        chunk_questions = questions_df.query(f'section_n_chunk=={chunk}')
        for _,q in chunk_questions.iterrows():

                section_output['question'].append(q.new_question)
                section_output['answer'].append(q['selected_ans'])

    output.append(section_output)
    
with open(dir+q_file_path) as f:
    text = f.read()
qs = text.replace('A:',' ').replace('\n','').replace('--------------------------------------------------','').split('Q:')

text = org_text
#text = "what year Israel was established? 1948"
try:
    nlp = spacy.load("en_core_web_lg")
except:
    #!pip install spacy-transformers
    !python -m spacy download en_core_web_lg
    nlp = spacy.load("en_core_web_lg")

with open(dir+full_q_file) as f:
    text_full = f.read()
text_full = [a.split('Text:') for a in text_full.split('\n\n\n\n')]


for i in range(max(questions_df.section_n)):
    section_output = {'question':[],'answer':[],'score':[]}
    for chunk in np.unique(questions_df.loc[questions_df.section_n==i,'section_n_chunk']):
        chunk_questions = questions_df.query(f'section_n_chunk=={chunk}')
        for _,q in chunk_questions.iterrows():

                section_output['question'].append(q.new_question)
                section_output['answer'].append(q['selected_ans'])

    output.append(section_output)


from Distractors import Distractors
find_dist = Distractors(org_text)
dist_out = find_dist.generate_distractors(output)
print(dist_out)


In [2]:
%pdb on

Automatic pdb calling has been turned ON


In [7]:
dir = '/home/ubuntu/Questions_generation/outputs/03_01_23_terror_cyber/'
output = []
import pandas as pd
import numpy as np
questions_df = pd.read_pickle(dir+'GQ+QA+GQ.pickle')
for i in range(max(questions_df.section_n)):
    section_output = {'question':[],'answer':[],'score':[]}
    for chunk in np.unique(questions_df.loc[questions_df.section_n==i,'section_n_chunk']):
        chunk_questions = questions_df.query(f'section_n_chunk=={chunk}')
        for _,q in chunk_questions.iterrows():

                section_output['question'].append(q.new_question)
                section_output['answer'].append(q['selected_ans'])

    output.append(section_output)

In [8]:
output

[{'question': ['What are the most recent attacks linked to Al Awlaki?',
   'What does the use of the internet for terrorist purposes do?',
   'What are some of the ways that terrorists use the internet?',
   'What terrorist group has been communicating online for years?',
   'What terrorist groups have communicated online for years?'],
  'answer': ['From september 11, 2001, to the shootings at fort hood in november 2009',
   'Online terrorism creates both challenges and opportunities in the endless war against terrorism',
   'Terrorists radicalize and recruit members, communicate and conspire, and plan and launch attacks online',
   'Al qaeda',
   'Al qaeda terrorist groups'],
  'score': []},
 {'question': ['The Popular Front for the Liberation of what country is a terrorist organization?',
   'What is the nationality of the ETA movement?',
   'What are the negatives of the use of the internet by terrorists?',
   'Who has become more sophisticated at exploiting online platforms?',
   '

In [1]:
import pandas as pd
out = pd.read_pickle('/home/ubuntu/Questions_generation/for_noam_2.pickle')
out

{0: {'texts': ['Background: Inflammatory bowel disease (IBD) comprises primarily the two disorders – ulcerative colitis and Crohn’s disease – that involve deregulated T cell responses. The ever-increasing incidence rate of Crohn’s disease and ulcerative colitis during recent decades, combined with the limited efficacy and potential adverse effects of current treatments, explain the real need for seeking more specific and selective methods for treating these diseases. Aim: To investigate the ability of interleukin 2 (IL2)– caspase 3 chimeric protein, designed to target activated T lymphocytes that express the high-affinity IL2 receptor, to ameliorate the clinical symptoms of acute murine experimental colitis, using a mouse model of dextran sodium sulfate (DSS)-induced colitis. Methods: Mice with DSS-induced colitis were treated with IL2–caspase 3 for 7 days and disease severity was assessed in parallel to control, non-treated mice, receiving only daily injections of phosphate-buffered s

In [2]:
questions_df['RQUGE'] = questions_df.apply(lambda x: mcq.rquge.scorer(x.text, x[questions_used], x.selected_ans)[0]['pred_score'] ,axis='columns')
questions_df = questions_df.sort_values('RQUGE',ascending=False).reset_index()
q_sim_mat,q_ans_sim_mat = mcq.find_similarity(questions_df[questions_used].to_list(),
                                            answers = (questions_df[questions_used]+' '+questions_df['selected_ans']).to_list())
filter_idx = mcq.filter_questions(questions_df[questions_used].to_list(),
                                q_sim_mat = q_sim_mat, ans_sim_mat = q_ans_sim_mat,
                                return_index=True)
questions_df['use_question']=False
questions_df.loc[filter_idx,'use_question']=True

,section_n,section_n_chunk,section_rank,text,question,question_ppl,answer_1,answer_2,answer_3,answer_4,short_answer_1,short_answer_2,short_answer_3,short_answer_4,generated_selected_ans,selected_ans,new_question
0,0,0.0,1.0,"Since the late 1980s, the Internet has proven ...",When did al awlaki join the terrorist group?,0.003643,"September 11, 2001,","September 11, 2001","From september 11, 2001, to the shootings at f...","From september 11, 2001, to the shootings at f...",9/11,2009,November 2009,2001,"from september 11, 2001, to the shootings at f...","From september 11, 2001, to the shootings at f...",What are the most recent attacks linked to Al ...
1,0,0.0,1.0,"Since the late 1980s, the Internet has proven ...",How does online terrorism affect the flow of i...,0.008470,Online terrorism creates both challenges and o...,Online terrorism creates both challenges and o...,Using and abusing the internet for terrorist p...,"Large or small, terrorist groups have their ow...",Large,The interactive capabilities of the internet,Terrorist groups have their own websites,Creating both challenges and opportunities in ...,online terrorism creates both challenges and o...,Online terrorism creates both challenges and o...,What does the use of the internet for terroris...
2,0,0.0,1.0,"Since the late 1980s, the Internet has proven ...",Why do terrorists use websites to spread propa...,0.017434,"Terrorists radicalize and recruit members, com...",They can now lure targeted individuals to the ...,They can now lure targeted individuals to the ...,"Large or small, terrorist groups have their ow...",Terrorists can now lure targeted individuals t...,"Raise funds, and seduce potential newcomers",Interactive capabilities of the internet,To assume a proactive position,"terrorists radicalize and recruit members, com...","Terrorists radicalize and recruit members, com...",What are some of the ways that terrorists use ...
3,0,0.0,1.0,"Since the late 1980s, the Internet has proven ...",What is the name of the group that has their o...,0.027545,Al qaeda,Al qaeda is a terrorist group.,The council on foreign relations concluded its...,"Al awlaki, a radical american muslim cleric of...",Jihadist,Terror,Qaeda,Terrorist groups,al qaeda,Al qaeda,What terrorist group has been communicating on...
4,0,0.0,1.0,"Since the late 1980s, the Internet has proven ...",Which terrorist group has their own websites?,0.069517,Al qaeda terrorist groups,Al qaeda,Al qaeda terrorist groups have their own websi...,"Al awlaki, a radical american muslim cleric of...",Jihadist,Large,Qaeda,Terrorist groups,al qaeda terrorist groups,Al qaeda terrorist groups,What terrorist groups have communicated online...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,5,5.1,1.0,The two men settled on a plan to bomb both sit...,The terrorists were arrested in december 2013 ...,0.002038,Suicide,Terrorism,Murder,Terrorist organizations,Terrorism,Terror,Terrorist organizations,Suicide truck bomb,terrorism,Terrorism,What is the increase in the use of social netw...
64,5,5.1,1.0,The two men settled on a plan to bomb both sit...,Who were abu sara and al-zawahiri?,0.004411,Two terrorist organizations,"During his questioning, abu sara told the isra...",Two men who decided to bomb both sites simulta...,Terrorist organizations,Militants,Terrorism,Terrorist organizations.,Two men,two men who decided to bomb both sites simulta...,Two men who decided to bomb both sites simulta...,Who were abu sara and al-zawahiri?
65,5,5.1,1.0,The two men settled on a plan to bomb both sit...,What was the first time abu sara was arrested?,0.020293,Two men settled on a plan to bomb both sites s...,Abu sara was arrested in december 2013 and dur...,In december 2013 they were arrested and during...,,Israeli,Israel,2012,2013,2012,2012,In what year did the bombings take place?
66,5,5.1,1.0,The two men settled on a plan to bomb both sit...,How many terrorist organizations were arrested?,0.021451,Two,One was the t

In [5]:
questions_df

,section_n,section_n_chunk,section_rank,text,question,question_ppl,answer_1,answer_2,answer_3,answer_4,short_answer_1,short_answer_2,short_answer_3,short_answer_4,generated_selected_ans,selected_ans,new_question
0,3,3.0,1.0,We analyze the storage and recall of factual a...,Inference for factors: rank-one model editing,0.000003,Model editing is feasible for factual associat...,An analysis of factual associations in transfo...,Inference for factors: rank-one model editing ...,Rank-one model editing (rome) is a method for ...,Factual association memory in transformer lang...,Model editing of factual associations in trans...,Model editing of factual associations in trans...,Rank-one model editing (rome),model editing is feasible for factual associat...,Model editing is feasible for factual associat...,What do our results suggest?
1,3,3.0,1.0,We analyze the storage and recall of factual a...,What is a factorial association?,0.000014,An analysis of factual associations in transfo...,Rank-one model editing (rome) is a method for ...,A causal intervention for identifying neuron a...,The storage and recall of factual associations...,Authors:,Factorial associations are localized computati...,Factual association,Factual association in transformer language mo...,the storage and recall of factual associations...,The storage and recall of factual associations...,What do we find evidence of?
2,3,3.0,1.0,We analyze the storage and recall of factual a...,Learning to recall factual associations in tra...,0.000935,Models for storing and recalling factual assoc...,Rank-one model editing (rome) is a model editi...,The storage and recall of factual associations...,Learning to recall factual associations in tra...,Model editing of transformer language models,Rank-one model editing (rome),The storage and recall of factual associations...,Learning to recall factual associations in tra...,the storage and recall of factual associations...,The storage and recall of factual associations...,What do we find evidence of?
3,3,3.0,1.0,We analyze the storage and recall of factual a...,Analyzing the storage and recall of factual as...,0.004359,An analysis of factual associations in transfo...,Analyzing the storage and recall of factual as...,Analyzing the storage and recall of factual as...,"In this paper, we analyze the storage and reca...",Authors:,Authors: baulab,Authors,Model editing of factual associations in trans...,analyzing the storage and recall of factual as...,Analyzing the storage and recall of factual as...,What do we analyze?
4,3,3.0,1.0,We analyze the storage and recall of factual a...,Model editing of factual associations in trans...,0.008617,Rank-one model editing (rome) is a model editi...,The storage and recall of factual associations...,Model editing of factual associations in trans...,Model editing of factual associations in trans...,Model experiment,Model-editing of factual associations in trans...,Rank-one model editing (rome),Model editing of factual associations in trans...,model editing of factual associations in trans...,Model editing of factual associations in trans...,What do we analyze?
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,24,24.0,1.0,Figure 9: Comparing mean causal traces across...,"For example, what are two examples of the same...",0.000352,This is a comparison of mean causal and salien...,Contextual examples of the same case are shown...,Figure 7: comparing mean causal traces across ...,The first example is a comparison of mean caus...,Visualizing the same cases as in figure 10:,Integrated gradients saliency maps,Visualizing the same cases as in figure 10,"Integrated gradients saliency maps, visualizin...",the first example is a comparison of mean caus...,The first example is a comparison of mean caus...,What is the final answer?
109,24,24.0,1.0,Figure 9: Comparing mean causal traces across...,"For example, what are two examples of the same...",0.000459,Contextual examples of the same case 